In [1]:
import os
import sys
import urllib, io
import pickle
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
proj_dir = os.path.abspath('../..')
import importlib

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))

import numpy as np
import scipy.stats as stats
import pandas as pd
from scipy import stats
from scipy.spatial import distance
from scipy import ndimage
from scipy.stats import entropy
from random import random
import itertools

from collections import Counter
import json
import re
import ast

from io import BytesIO
import base64

from PIL import Image, ImageOps, ImageDraw, ImageFont 
import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from matplotlib import colors

import plotly
import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

from collections import Counter

# import drawing_utils as drawing 
# import block_construction_helpers as bc 

sys.path.append(os.path.join(proj_dir,'stimuli/block_utils'))
import blockworld_utils as utils
import domino_settings as dominoes

In [10]:
# setup

four_block_towers = [
    'wa',
    'wb',
    'wc',
    'wd',
    'ta',
    'tb',
    'tc',
    'td'
]

ambiguous_stims = ['wa&wb-tc&td', 'wc&wd-ta&tb']

conditions = {'A': {'wide': ['wa','wb'],
                    'tall': ['ta','tb'],
                    'ambiguous': ['wa&wb-tc&td','wc&wd-ta&tb']},
              'B': {'wide': ['wc','wd'],
                    'tall': ['tc','td'],
                    'ambiguous': ['wa&wb-tc&td','wc&wd-ta&tb']}}


In [ ]:
# distractor towers (alternative towers that don't (together?) reconstruct a learned visual chunk)




In [ ]:
trials

In [29]:
# generate synthetic data

WIDE_DISTRACTOR = 'WIDE_DISTRACTOR'
TALL_DISTRACTOR = 'TALL_DISTRACTOR'

df_trials = pd.DataFrame()

for c_i, (condition_name, condition_dict) in enumerate(conditions.items()):
    
    # construct trials 
    wides = condition_dict['wide']
    talls = condition_dict['tall']
    ambiguous = condition_dict['ambiguous']
    
    for ambiguous_stim in ambiguous:
        wide_parts = ambiguous_stim.split('-')[0].split('&')
        tall_parts = ambiguous_stim.split('-')[1].split('&')
        
        # if this ambiguous stim can be constructed from wides in condition
        if all([part in wides for part in wide_parts]):
            
            #construct trained trials (wide)
            valid_trained_trial = {
                'condition': condition_name,
                'ambiguous': ambiguous_stim,
                'part_a': wide_parts[0],
                'part_b': wide_parts[1],
                'part_type': 'wide',
                'valid': True,
                'trained': True,
            }
            
            
            invalid_trained_trial = {
                'condition': condition_name,
                'ambiguous': ambiguous_stim,
                'part_a': WIDE_DISTRACTOR,
                'part_b': WIDE_DISTRACTOR,
                'part_type': 'wide',
                'valid': False,
                'trained': True,
            }
            
            
            #construct untrained trials (tall)
            valid_untrained_trial = {
                'condition': condition_name,
                'ambiguous': ambiguous_stim,
                'part_a': tall_parts[0],
                'part_b': tall_parts[1],
                'part_type': 'tall',
                'valid': True,
                'trained': False,
            }
            
            invalid_untrained_trial = {
                'condition': condition_name,
                'ambiguous': ambiguous_stim,
                'part_a': TALL_DISTRACTOR,
                'part_b': TALL_DISTRACTOR,
                'part_type': 'tall',
                'valid': False,
                'trained': False
            }
            
        elif all([part in talls for part in tall_parts]):
            
            #construct trained trials (tall)
            valid_trained_trial = {
                'condition': condition_name,
                'ambiguous': ambiguous_stim,
                'part_a': tall_parts[0],
                'part_b': tall_parts[1],
                'part_type': 'tall',
                'valid': True,
                'trained': True
            }
            
            invalid_trained_trial = {
                'condition': condition_name,
                'ambiguous': ambiguous_stim,
                'part_a': TALL_DISTRACTOR,
                'part_b': TALL_DISTRACTOR,
                'part_type': 'tall',
                'valid': False,
                'trained': True,
            }
            
            #construct untrained trials (wide)
            valid_untrained_trial = {
                'condition': condition_name,
                'ambiguous': ambiguous_stim,
                'part_a': wide_parts[0],
                'part_b': wide_parts[1],
                'part_type': 'wide',
                'valid': True,
                'trained': True,
            }
            
            
            invalid_untrained_trial = {
                'condition': condition_name,
                'ambiguous': ambiguous_stim,
                'part_a': WIDE_DISTRACTOR,
                'part_b': WIDE_DISTRACTOR,
                'part_type': 'wide',
                'valid': False,
                'trained': False
            }
            
            
        else:
            print('problem: ambiguous stim not constructable with talls or wides')
            
        df_trials = df_trials.append([valid_trained_trial, 
                          invalid_trained_trial, 
                          valid_untrained_trial,
                          invalid_untrained_trial], ignore_index=True)
    
df_trials

,condition,ambiguous,part_a,part_b,part_type,valid,trained
0,A,wa&wb-tc&td,wa,wb,wide,True,True
1,A,wa&wb-tc&td,WIDE_DISTRACTOR,WIDE_DISTRACTOR,wide,False,True
2,A,wa&wb-tc&td,tc,td,tall,True,False
3,A,wa&wb-tc&td,TALL_DISTRACTOR,TALL_DISTRACTOR,tall,False,False
4,A,wc&wd-ta&tb,ta,tb,tall,True,True
5,A,wc&wd-ta&tb,TALL_DISTRACTOR,TALL_DISTRACTOR,tall,False,True
6,A,wc&wd-ta&tb,wc,wd,wide,True,True
7,A,wc&wd-ta&tb,WIDE_DISTRACTOR,WIDE_DISTRACTOR,wide,False,False
8,B,wa&wb-tc&td,tc,td,tall,True,True
9,B,wa&wb-tc&td,TALL_DISTRACTOR,TALL_DISTRACTOR,tall,False,True


#### Notes to self

__
This assumes 2 ambiguous stimuli, built in both conditions.
Condition A builds the first ambigous stim with wides, and the second with talls. 
Condition B builds the first with talls, and second with wides.
This means each condition has 2 wides and 2 talls to learn during assembly.

This is the minimum amount of stimuli. This means it will be singular, so we'll need to add more trials.

We could simply double the design- 4 ambiguous stims; 4 wide and 4 tall chunks each. Checking for overlap should be fairly easy. 

__
We've added the minimum number of trials here, which means people only see two kinds of ambiguous stim!
That's not enough-- there are probably going to be learning effects.






__

We could add a single 4-block structure to each training condition, construct new ambiguous stims.
Rejection sampling might be the best method anyway:
- add n 4-block towers to each type, for each condition
- construct all ambiguous stims
- 

I want to keep this as a backup option.






In [33]:
# data columns
['condition', 'ambiguous', 'part_a', 'part_b', 'part_type', 'valid', 'trained']\ # metadata
+ ['ppt', 'trial_num', 'RT', 'correct', 'stim_repetition'] # trial data

#'stim_repetition': number time that this one has been presented.
# Checking for learning effects of a particular ambiguous stim

['condition',
 'ambiguous',
 'part_a',
 'part_b',
 'part_type',
 'valid',
 'trained',
 'ppt',
 'trial_num',
 'RT',
 'correct',
 'stim_repetition']